In [2]:
#Set up the notebook environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr
from scipy import signal as sig
import xgboost as xgb
from utils import *
from xgboost import XGBRegressor

In [3]:
raw = scipy.io.loadmat('./datasets/leaderboard_data.mat')
ecog_1 = raw['leaderboard_ecog'][0][0]
ecog_2 = raw['leaderboard_ecog'][1][0]
ecog_3 = raw['leaderboard_ecog'][2][0]

In [4]:
winLen = 100 / 1e3
winOverlap = 50 / 1e3
winDisp = winLen - winOverlap
NumWins(ecog_1, 1000, winLen, winDisp)

feature_1 = get_windowed_feats(ecog_1, 1000, winLen, winOverlap)
# R_1 = create_R_matrix(feature_1, 5)
feature_2 = get_windowed_feats(ecog_2, 1000, winLen, winOverlap)
# R_2 = create_R_matrix(feature_2, 5)
feature_3 = get_windowed_feats(ecog_3, 1000, winLen, winOverlap)
# R_3 = create_R_matrix(feature_3, 5)

idx_1 = np.load('./models/idx_S1.npy')
idx_2 = np.load('./models/idx_S2.npy')
idx_3 = np.load('./models/idx_S3.npy')

R_1 = create_R_matrix(feature_1, 20)[:, idx_1]
R_2 = create_R_matrix(feature_2, 20)[:, idx_2]
R_3 = create_R_matrix(feature_3, 20)[:, idx_3]

R_list = [R_1, R_2, R_3]

/usr/local/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 62, using nperseg = 62
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 48, using nperseg = 48
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 64, using nperseg = 64
  warnings.warn('nperseg = {0:d} is greater than input length '


In [5]:
train_mean_S1 = np.load('./models/train_mean_S1.npy')
train_std_S1 = np.load('./models/train_std_S1.npy')

train_mean_S2 = np.load('./models/train_mean_S2.npy')
train_std_S2 = np.load('./models/train_std_S2.npy')

train_mean_S3 = np.load('./models/train_mean_S3.npy')
train_std_S3 = np.load('./models/train_std_S3.npy')

train_mean_ls = [train_mean_S1, train_mean_S2, train_mean_S3]
train_std_ls = [train_std_S1, train_std_S2, train_std_S3]

In [6]:
device = torch.device("cpu")

In [1]:
import lightgbm
lightgbm.Booster(model_file=f'./models/lgbr_f{0}_S{1}.txt')

In [7]:
predictions = []
for i in range(3):
    # Load XGB
    xgb_reg = xgb.XGBRegressor()
    xgb_reg.load_model(f"./models/XGB_S{i + 1}.json")

    prediction_xgb = xgb_reg.predict(R_list[i])
    
    # Load LGBM
    lgbm_reg_list = [lightgbm.Booster(model_file=f'./models/lgbr_f{j}_S{i + 1}.txt') for j in range(4)]
    
    prediction_lgbm_list = [lgbm_reg.predict(R_list[i]) for lgbm_reg in lgbm_reg_list]
    prediction_lgbm = np.vstack(prediction_lgbm_list).T
    
#     Load NN
    leaderboard_dataset = FingerFeatureDataset((R_list[i] - train_mean_ls[i]) / train_std_ls[i], np.zeros(R_list[i].shape[0]).copy())
    dataloader = DataLoader(leaderboard_dataset, batch_size=16, shuffle=False)

    net = FingerRegressor(R_list[i].shape[1], 4).to(device)
    net.load_state_dict(torch.load(f'./models/NN_S{i + 1}.pth', map_location=device))
    
    with torch.no_grad():
        pred = []
        net.eval()
        for i, (ecog, dg) in enumerate(dataloader):
            ecog = ecog.to(device)
            dg = dg.to(device)
            output = net(ecog).to(device)
            pred += [output.detach().cpu().numpy()]

    prediction_NN = np.concatenate(pred)
    
    
    prediction = (prediction_lgbm + prediction_NN) / 2
    predictions.append(prediction)

In [8]:
pack_submission(predictions, 'leaderboard_prediction.mat', 'predicted_dg')

In [9]:
raw = scipy.io.loadmat('./leaderboard_prediction.mat')
raw['predicted_dg'].shape

(3, 1, 147500, 5)